In [ ]:
import pandas as pd
from duckdb import query

In [ ]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
pd.set_option('mode.chained_assignment', None)

## Join Tables

In [ ]:
# these files are the output of Tyndale/code/01. Tyndale txt to pkl.py
# which takes the txt files as its input in Tyndale/data
# i'm going to join them all together and clean things up, add some new columns, etc.
# then make pkl files for django models
a = pd.read_pickle("../data/tagnt.pkl")
b = pd.read_pickle("../data/strongs.pkl")
c = pd.read_pickle("../data/tegmc.pkl")
d = pd.read_pickle("../data/bcvIndex.pkl")

# lexnalt data from biblehub
e = pd.read_pickle('../../pickles/lexnalt.pkl')

In [ ]:
# see what's in each table, define fields to keep going forward
print('tagnt columns: ',a.columns)
print('strongs columns: ',b.columns)
print('tegmc columns: ',c.columns)
print('bcvIndex columns: ',d.columns)
tagnt_fields = ['bcv','greek','english', 'strongs_id', 'morphology_id',]
strongs_fields = ['strongs', 'lexicon', 'gloss', 'transliteration','definitionhtml',]
tegmc_fields = ['morphology','function', 'tense','voice','mood','person','case','gender','number'] 
bcvIndex_fields = ['bcv','book','bookLong']

In [ ]:
## not perfect, but forcing strongs ids to join 1-to-1 to strongs table
## some of the words had two or more strongs ids associated with them,
## so i'm keeping the first one in that case
a['strongsidlength'] = a.strongs_id.apply(lambda x: len(x))
print(a.groupby(['strongsidlength']).count().reset_index()[['strongsidlength','greek']])
a['strongs_id'] = a.strongs_id.apply(lambda x: x.split('+')[0] if len(x) > 6 else x)
a.sort_values('strongsidlength',ascending=False)
a['strongsidlength'] = a.strongs_id.apply(lambda x: len(x))
print(a.groupby(['strongsidlength']).count().reset_index()[['strongsidlength','greek']])


In [ ]:
fixlist = ['CONJ +P-1NS', 'PRT-N +PRT', 'CONJ +ADV', 'CONJ +I-NSN', 'CONJ +D-NPN', 'PRT-N +ADV', 'CONJ +COND', 'CONJ +D-APN',
'CONJ +D-ASM', 'PREP+N-ASN', 'CONJ +D-NPM', 'CONJ +P-1DS', 'CONJ +D-NSM', 'A-NPM-NUI +A-NPM-NUI', 'A-APN-NUI +A-APN-NUI',
'CONJ +PRT +PRT', 'CONJ +P-1AS', 'ADV +V-AAP-NSM', 'CONJ +PRT', 'CONJ +D-APM', 'COND +CONJ', 'PREP +ADV', 'PRT +ADV', 'ADV +N-NPF']

a['morphology_id'] = a.morphology_id.apply(
    lambda x: 'CONJ' if x in fixlist and x[:4]=='CONJ' else
              'CONJ' if x in fixlist and x[:4]=='COND' else
              'PRT' if x in fixlist and x[:3]=='PRT' else
              'PREP' if x in fixlist and x[:4]=='PREP' else
              'A-NPM-NUI' if x in fixlist and x[:9]=='A-NPM-NUI' else
              'A-APN-NUI' if x in fixlist and x[:9]=='A-APN-NUI' else
              'ADV' if x in fixlist and x[:3]=='ADV' else x
)

In [ ]:
# join tables
df = a[a.nestleAland==True][tagnt_fields]\
    .reset_index(drop=True)\
    .rename(columns={'strongs_id':'strongs', 'morphology_id':'morphology'})\
    .merge(b[strongs_fields], how='left', on='strongs')\
    .merge(c[tegmc_fields], how='left', on='morphology')\
    .merge(d[bcvIndex_fields], how='left', on='bcv').fillna('')
print("shape: ", df.shape)
df.head(1)


In [ ]:
# renaming these to xxxxxx2 because i'm going to update each field subsequently
df = df.rename(columns={
    'function': 'function2',
    'tense': 'tense2',
    'voice': 'voice2',
    'mood': 'mood2',
    'person': 'person2',
    'case': 'case2',
    'gender': 'gender2',
    'number': 'number2',
})

## Add some ID columns

In [ ]:
# build some id columns
bcv = list(df.bcv)
bcvw = ['01010101']
i = 1
bcv_ = '010101'
for item in bcv[1:]:
    if item == bcv_:
        i += 1
    else:
        i = 1
    w = ('0' + str(i))[-2:]
    bcvw.append(item+w)
    bcv_ = item

df['bcvw'] = bcvw
df['bk'] = df.bcvw.apply(lambda x: x[:2])
df['ch'] = df.bcvw.apply(lambda x: x[2:4])
df['vs'] = df.bcvw.apply(lambda x: x[4:6])
df['wd'] = df.bcvw.apply(lambda x: x[6:8])
df['bc'] = df['bk'] + df['ch']

id_columns = ['book','bcvw','bcv','bc','bk','ch','vs','wd']
df = df[id_columns + [col for col in list(df.columns) if col not in id_columns]]

## Join word frequency

In [ ]:
# get the word freq for each strongs number
strongs_count = query(f"""
SELECT 
    strongs,
    lexicon,
    gloss,
    transliteration,
    count(*) as lexicon_freq_nt 
FROM df
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4
""").to_df().sort_values(['lexicon_freq_nt','strongs'],ascending=[False,True]).reset_index(drop=True).reset_index().rename(columns={'index':'lexicon_id_int'})

In [ ]:
strongs_count.head(2)

In [ ]:
df = df.merge(strongs_count[['strongs', 'lexicon_freq_nt', 'lexicon_id_int']], how='left', on='strongs')

In [ ]:
df['lexicon_id'] = df.lexicon_id_int.apply(lambda x: ('000'+str(x))[-4:] )
df.head(2)

## Redefine/Collapse/Simplify some variables 

In [ ]:
df['function'] = df.function2.apply(lambda x:
    'ADJ' if x in ['Adjective',] else
    'ADV' if x in ['Adverb', 'Adverb or adverb and particle combined',] else
    'ART' if x in ['Definite article',] else
    'CONJ' if x in ['Conjunction'] else
    'PRON' if x in [
        'Correlative or Interrogative pronoun',
        'Correlative pronoun',
        'Demonstrative pronoun',
        'Indefinite pronoun',
        'Interrogative pronoun',
        'Personal pronoun',
        'Possessive pronoun',
        'Reciprocal pronoun',
        'Reflexive pronoun',
        'Relative pronoun',
    ] else
    'PRT' if x in ['Interrogative Particle', 'Negative Particle', 'Particle or Disjunctive','Interjection',] else
    'PREP' if x in ['Preposition',] else
    'NOUN' if x in ['Noun',] else
    'V' if x in ['Verb',] 
    else 'OTHER'    
)
df.groupby(['function','function2']).count()[['wd']]


In [ ]:
query("select function, count(*) as count from df group by 1 order by count desc").to_df()

In [ ]:
## at one point, was attempting to make consistent functions by lexicon_id, but not sure that's desired on frontend

# a = (df.groupby(['lexicon_id','function']).count().reset_index()[['lexicon_id','function', 'wd']]).groupby('lexicon_id').count().reset_index()
# dups = list(a[a.wd>0].lexicon_id)
# dups
# a = df[df.lexicon_id.isin(dups)].groupby(['lexicon_id','function']).count().reset_index()[['lexicon_id', 'function', 'wd']]
# b = a.groupby('lexicon_id').max().reset_index()[['lexicon_id','wd']]
# b['flag'] = 1
# c = a.merge(b,how='left', on=['lexicon_id', 'wd'])
# d = c[c.flag==1].drop(columns=['flag','wd',]).rename(columns={'function':'function_new'})

# temp = df.merge(d, how='left', on='lexicon_id')
# e = temp[(temp.function != temp.function_new)].sort_values('lexicon_id', ascending=False)
# e.groupby(['lexicon_id','function_new']).count()

# temp[temp.lexicon_id.isin(['1254', '3195', '3485'])]
# temp['function_final'] = temp.apply(lambda x: 'ADV' if x.strongs=='G1622' else 'ADV' if x.strongs=='G2115' else '')


In [ ]:
df['case'] = df.case2.apply(lambda x:
'N' if x == 'Nominative' else
'G' if x == 'Genitive' else
'D' if x == 'Dative' else
'A' if x == 'Accusative' else
'V' if x == 'Vocative' else x
)
df.groupby(['case','case2']).count()[['wd']]

In [ ]:
df['tense'] = df.tense2.apply(lambda x:
'P' if x in ['Present',] else
'F' if x in ['Future', '2nd Future',] else
'I' if x in ['Imperfect',] else
'R' if x in ['Perfect', '2nd Perfect', 'Pluperfect', '2nd Pluperfect',] else
'A' if x in ['Aorist',] else
'2A' if x in ['2nd Aorist',]
else x)
df.groupby(['tense','tense2']).count()[['wd']]

In [ ]:
df['gender'] = df.gender2.apply(lambda x: 'F' if x=='Feminine' else 'M' if x=='Masculine' else 'N' if x=='Neuter' else x)
df.groupby(['gender', 'gender2']).count()[['wd']]

In [ ]:
df['person'] = df.person2.apply(lambda x: 
'1' if x in ['1st', '1st Plural', '1st Singular',] else 
'2' if x in ['2nd', '2nd Singular'] else
'3' if x in ['3rd']
else x)
df.groupby(['person', 'person2']).count()[['wd']]

In [ ]:
df['voice'] = df.voice2.apply(lambda x:
'A' if x in ['Active'] else
'M' if x in ['Middle', 'Middle Deponent',] else
'P' if x in ['Passive', 'Middle or Passive', 'Passive Deponent', 'Middle or Passive Deponent',]
else x
)
df.groupby(['voice', 'voice2']).count()[['wd']]

In [ ]:
df['mood'] = df.mood2.apply(lambda x:
'I' if x in ['Indicative','Optative',] else
'M' if x in ['Imperative',] else
'S' if x in ['Subjunctive',]
else x
)
df.groupby(['mood', 'mood2']).count()[['wd']]

In [ ]:
df['number'] = df.number2.apply(lambda x: 'S' if x == 'Singular' else 'P' if x == 'Plural' else x)
df.groupby(['number', 'number2']).count()[['wd']]

In [ ]:
df = df[['bookLong', 'book', 'bcvw', 'bcv', 'bc', 'bk', 'ch', 'vs', 'wd',
       'greek', 'english', 'strongs', 'lexicon_id', 'lexicon_id_int', 'morphology', 'lexicon', 'gloss', 'transliteration',
       'function', 'tense', 'voice', 'mood', 'person', 'case', 'gender', 'number', 'lexicon_freq_nt', 'definitionhtml',]]

## Understand parsing breakouts, and Add a new Parsing field

In [ ]:
df.head(2)

In [ ]:
df[df.function.isin(['ADV', 'PREP', 'CONJ', 'PRT', 'OTHER'])].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

In [ ]:
df[df.function.isin(['ART', 'ADJ', 'NOUN'])].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

In [ ]:
df[df.function=='PRON'].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

In [ ]:
# assign mood for infinitives & participles
df['mood'] = df.apply(lambda x: 'N' if (x.function=='V') and (x.mood=='') and (x.person=='') and (x.case=='') and (x.gender=='') and (x.number=='') else x.mood, axis=1)
df['mood'] = df.apply(lambda x: 'P' if (x.function=='V') and (x.case != "") else x.mood, axis=1)

In [ ]:
df[df.function=='V'].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

In [ ]:
## Parsing
df['parsing'] = df.apply(lambda x:
    (x.function + '-' + x.case + x.gender + x.number) if x.function in ['ART', 'ADJ', 'NOUN'] else
    (x.function + '-' + x.case + x.gender + x.number) if ((x.function=='PRON') & (x.person not in ['1', '2', '3'])) else
    (x.function + '-' + x.person + x.number + '-' + x.case + x.gender) if x.function=='PRON' else
    (x.function + '-' + x.tense + x.voice + x.mood) if x.mood == 'N' else
    (x.function + '-' + x.tense + x.voice + x.mood + '-' + x.case + x.gender + x.number) if x.mood == 'P' else
    (x.function + '-' + x.tense + x.voice + x.mood + '-' + x.person + x.number) if x.function == 'V'    
    else x.function
,axis=1)

In [ ]:
parsing = df.groupby(['parsing']).count()['wd']

In [ ]:
df = df[['bcv', 'bookLong', 'book', 'bk', 'ch', 'vs', 'wd', 'greek', 'english', 'parsing', 'lexicon', 'gloss', 'transliteration','lexicon_freq_nt', 'strongs', 'definitionhtml','lexicon_id', 'lexicon_id_int', 'function', 'tense', 'voice', 'mood', 'person', 'case', 'gender', 'number']]
df.head(2)

## Split word and punctuation for Greek and English

In [ ]:
# note: this function can return all characters for a language, 
# but using it right now to get unique characters that end
# a word, so I can separate out the puncuation characters
def get_unique_characters(var):
    word_string=''
    for word in df[var].to_list():
        word_string+=word[-1]
    unique_letters = []
    for letter in word_string:
        if letter not in unique_letters:
            unique_letters.append(letter)
    return sorted(unique_letters)

In [ ]:
greek_chars = get_unique_characters('greek')
print(greek_chars[:])
english_chars = get_unique_characters('english')
print(english_chars[:])

In [ ]:
# determined these lists from the output above, will use in next step
greek_puncs = [',', '.', '¶', 'ͅ', ';', '·',]
english_puncs = [' ', '!', '"', ',', '.', ':', ';', '?',]

In [ ]:
greek_without_puncuation = []
greek_puncuation = []
for word in list(df['greek']):    
    if len(word)>=3 and word[-3] in greek_puncs and word[-2] in greek_puncs and word[-1] in greek_puncs:            
        greek_puncuation.append(word[-3:])
        greek_without_puncuation.append(word[:-3])
    elif len(word)>=2 and word[-2] in greek_puncs and word[-1] in greek_puncs:            
        greek_puncuation.append(word[-2:])
        greek_without_puncuation.append(word[:-2])
    elif word[-1] in greek_puncs:            
        greek_puncuation.append(word[-1:])
        greek_without_puncuation.append(word[:-1])        
    else:
        greek_puncuation.append('')
        greek_without_puncuation.append(word)

df['greek2'] = greek_without_puncuation
df['greek_punc'] = greek_puncuation

In [ ]:
english_without_puncuation = []
english_puncuation = []
for word in list(df['english']):    
    if len(word)>=3 and word[-3] in english_puncs and word[-2] in english_puncs and word[-1] in english_puncs:            
        english_puncuation.append(word[-3:])
        english_without_puncuation.append(word[:-3])
    elif len(word)>=2 and word[-2] in english_puncs and word[-1] in english_puncs:            
        english_puncuation.append(word[-2:])
        english_without_puncuation.append(word[:-2])
    elif word[-1] in english_puncs:            
        english_puncuation.append(word[-1:])
        english_without_puncuation.append(word[:-1])        
    else:
        english_puncuation.append('')
        english_without_puncuation.append(word)

df['english2'] = english_without_puncuation
df['english_punc'] = english_puncuation

In [ ]:
df = df.rename(columns={'greek2': 'greek', 'greek':'greek2', 'english2':'english', 'english':'english2'})

In [ ]:
verses = list(df['bcv'].unique())
verse_dict = {v:i for i,v in enumerate(verses,1)}
df['vs_id'] = df.bcv.apply(lambda x: verse_dict[x])

In [ ]:
df = df[[
 'vs_id',
 'bcv',
 'bookLong',
 'book',
 'bk',
 'ch',
 'vs',
 'wd',
 'greek',
 'english',
 'parsing',
 'lexicon',
 'lexicon_freq_nt',
 'strongs',
 'definitionhtml',
 'lexicon_id',
 'lexicon_id_int',
 'gloss',
 'transliteration',
 'function',
 'tense',
 'voice',
 'mood',
 'person',
 'case',
 'gender',
 'number', 
 'greek_punc',
 'english_punc',]]
df.head(2)

## Get Paradigms

In [ ]:
# grouping by parsing fields
# because of various accents, keeping max count when duplicates occur
# note: lexicon_id_int parameter is ordering of strongs ids by word frequency, so 0 is the article bc it's most common
def get_paradigms(lexicon_id_int):  
    fields = ['lexicon_id', 'lexicon_id_int', 'parsing',]
    w = df[df.lexicon_id_int==lexicon_id_int]
    w['greek'] = w.greek.apply(lambda x: x.lower())
    x = w.groupby(fields+['greek']).count().reset_index()[fields+['greek', 'wd']].reset_index()
    x['wd2'] = x['wd'] + x['index']/10000
    y = x.groupby(fields).max().reset_index()[fields+['wd2']]
    y2 = x.groupby(fields).sum().reset_index()[fields+['wd']]
    y['flag'] = 1
    z = x.merge(y, how='left', on=fields+['wd2']).drop(columns=['wd','wd2',])
    z = z[z.flag==1].drop(columns=['flag',])
    z = z.merge(y2, how='left', on=fields).drop(columns=['index', 'lexicon_id_int']).rename(columns={'wd':'paradigm_freq_nt'})

    # run this to see what decisions are made in this function --> z,d = get_paradigms(18)
    # print(w.shape[0])
    # print(z.wd.sum())
    
    return z


In [ ]:
z = get_paradigms(18)
z.head(2)

In [ ]:
# step takes about 45sec
lexicon_ids_int = sorted(list(df['lexicon_id_int'].unique()))
dfs = []
for lexicon_id_int in lexicon_ids_int:
    dfs.append(get_paradigms(lexicon_id_int))
pdgm = pd.concat(dfs).reset_index(drop=True)


In [ ]:
# make sure that worked...
# the difference should be exactly zero, but note decisions made for duplicate parsing records in fxn above
fields = ['lexicon_id', 'parsing',]
rows_in_gnt = df.shape[0]
paradigm_word_count = pdgm.groupby(fields).sum().paradigm_freq_nt.sum()
print(rows_in_gnt - paradigm_word_count)

## Create models for database

In [ ]:
# paradigm model
# pdgm['pdgm_id'] = pdgm.lexicon_id + '_' + pdgm.parsing
pdgm = pdgm.rename(columns={'lexicon_id': 'pdgm_lexn_id', 'parsing':'pdgm_pars_id', 'greek':'pdgm_greek', 'paradigm_freq_nt':'pdgm_freq_nt'})
pdgm['pdgm_id'] = pdgm.pdgm_lexn_id + "_" + pdgm.pdgm_pars_id
pdgm = pdgm[['pdgm_id', 'pdgm_greek', 'pdgm_freq_nt', 'pdgm_lexn_id', 'pdgm_pars_id',]]
pdgm.head()

In [ ]:
# parsing model
parsing_fields = ['parsing', 'function', 'tense', 'voice','mood', 'person', 'case', 'gender', 'number']
pars = df.groupby(parsing_fields).count()['wd'].reset_index().sort_values('parsing').rename(columns={'wd':'parsing_freq_nt'})
pars = pars.rename(columns={'parsing':'pars_id', 'function':'pars_function', 'tense':'pars_tense', 'voice':'pars_voice', 'mood':'pars_mood', 'person':'pars_person', 'case':'pars_case', 'gender':'pars_gender', 'number':'pars_number', 'parsing_freq_nt':'pars_freq_nt'})

# sorting for frontend to order paradigms easier
cols = list(pars.columns)
rank_function = {'NOUN':1, 'PRON':2, 'ADJ':3, 'V':4, 'ART':5, 'ADV':6, 'CONJ':7, 'PREP':8, 'PRT':9}
rank_tense = {'P':1, 'I':2, 'F':3, 'A':4, '2A':5, 'R':6, '':7}
rank_voice = {'A':1, 'M':2, 'P':3, '':4}
rank_mood = {'I':1, 'M':2, 'N':3, 'S':4, 'P':5, '':6}
rank_person = {'1':1, '2':2, '3':3, '':4}
rank_case = {'N':1, 'G':2, 'D':3, 'A':4, 'V':5, '':6}
rank_gender = {'M':1, 'F':2, 'N':3, '':4}
rank_number = {'S':1, 'P':2, '':3}

pars['rank_function'] = pars.pars_function.apply(lambda x: rank_function[x])
pars['rank_tense'] = pars.pars_tense.apply(lambda x: rank_tense[x])
pars['rank_voice'] = pars.pars_voice.apply(lambda x: rank_voice[x])
pars['rank_mood'] = pars.pars_mood.apply(lambda x: rank_mood[x])
pars['rank_person'] = pars.pars_person.apply(lambda x: rank_person[x])
pars['rank_case'] = pars.pars_case.apply(lambda x: rank_case[x])
pars['rank_gender'] = pars.pars_gender.apply(lambda x: rank_gender[x])
pars['rank_number'] = pars.pars_number.apply(lambda x: rank_number[x])

pars = pars.sort_values(by=[
    'rank_function',
    'rank_mood',
    'rank_tense',
    'rank_voice', 
    'rank_number',   
    'rank_person',    
    'rank_gender',
    'rank_case'
])

pars = pars.drop(columns=['rank_function', 'rank_tense', 'rank_voice', 'rank_mood', 'rank_person', 'rank_case',
'rank_gender', 'rank_number']).reset_index(drop=True).reset_index().rename(columns={
    'index':'pars_rank'
})

pars = pars[cols + ['pars_rank']]

pars.head()





In [ ]:
# lexicon model
lexicon_fields = ['lexicon_id', 'lexicon', 'gloss', 'transliteration', 'lexicon_freq_nt','strongs', 'definitionhtml',]
lexn = df.groupby(lexicon_fields).count()['wd'].reset_index().sort_values('lexicon_id').drop(columns=['wd',])

lexn = lexn.rename(columns={'lexicon_id':'lexn_id', 'lexicon':'lexn_greek', 'gloss':'lexn_gloss', 
'transliteration':'lexn_transliteration', 'lexicon_freq_nt':'lexn_freq_nt', 'strongs':'lexn_strongs', 'definitionhtml':'lexn_dictionaryhtml'})

lexn['lexn_strongs'] = lexn.lexn_strongs.apply(lambda x: x[1:])
temp = pdgm[['pdgm_lexn_id', 'pdgm_pars_id']].rename(columns={'pdgm_lexn_id':'lexn_id'})
temp['lexn_function'] = temp.pdgm_pars_id.apply(lambda x: x.split('-')[0])
temp2 = temp.groupby(['lexn_id',]).max().reset_index()[['lexn_id', 'lexn_function']]

lexn = lexn.merge(temp2, how='left', on='lexn_id')

# join lexnalt data from biblehub
temp = pd.read_pickle('../../pickles/lexnalt.pkl')
# temp['lexn_strongs'] = temp.strongs.apply(lambda x: ('000'+str(x))[-4:])

lexn = lexn.merge(temp, how='left', on='lexn_strongs')

lexn['greek'] = lexn.greek.fillna('')
lexn['definition'] = lexn.definition.fillna('')
lexn['usage'] = lexn.usage.fillna('')
lexn['lexn_greek_long'] = lexn.apply(lambda x: x.greek if x.greek != '' else x.lexn_greek, axis=1)
lexn['lexn_definition'] = lexn.apply(lambda x: x.definition if x.definition != '' else x.lexn_gloss, axis=1)
lexn['lexn_usage'] = lexn.apply(lambda x: x.usage if x.usage != '' else x.lexn_gloss, axis=1)
lexn['lexn_definition'] = lexn.lexn_definition.apply(lambda x: x.split('NAS Exhaustive')[0].split("Thayer's Greek")[0])
lexn['lexn_usage'] = lexn.lexn_usage.apply(lambda x: x.split('NAS Exhaustive')[0].split("Thayer's Greek")[0][:-1])
lexn = lexn[[
    'lexn_id',
    'lexn_greek',
    'lexn_greek_long',
    'lexn_transliteration',
    'lexn_gloss',
    'lexn_definition',    
    'lexn_usage',    
    'lexn_strongs',
    'lexn_function',
    'lexn_freq_nt',
]]
lexn.head(5)


In [ ]:
df['word_id'] = df.bcv + df.wd
df['word_book_id'] = df.bk
df['word_chap_id'] = df.bk + df.ch
df['word_vers_id'] = df.bcv
df.head(1)

In [ ]:
# book model
book = df.groupby(['bk', 'book', 'bookLong']).count()['wd'].reset_index()
num_chapters_per_book = df.groupby(['bk']).max()['ch'].reset_index().rename(columns={'ch':'book_num_chapters'})
temp = df.groupby(['bk','ch','vs']).count()['wd'].reset_index()
num_verses_per_book = temp.groupby(['bk']).count()['wd'].reset_index().rename(columns={'wd':'book_num_verses'})
book = book\
        .merge(num_chapters_per_book, how='left', on='bk')\
        .merge(num_verses_per_book, how='left', on='bk')\
        .rename(columns={'bk':'book_id', 'bookLong':'book_name', 'book':'book_name_abbrev', 'book_num_chapters':'book_num_chapters', 'book_num_verses':'book_num_verses', 'wd':'book_num_words'})

book = book[['book_id', 'book_name', 'book_name_abbrev', 'book_num_chapters', 'book_num_verses', 'book_num_words']]   
book


In [ ]:
# word model
word = df[['word_id', 'greek', 'english', 'greek_punc', 'english_punc', 'parsing', 'lexicon_id', 'word_book_id', 'word_chap_id', 'word_vers_id', ]]
word = word.rename(columns={'greek':'word_greek', 'english':'word_english', 'greek_punc':'word_greek_punc', 'english_punc':'word_english_punc', 'parsing':'word_pars_id', 'lexicon_id':'word_lexn_id', })
word['word_chap_num'] = word.word_chap_id.apply(lambda x: int(x[2:]))
word['word_vers_num'] = word.word_vers_id.apply(lambda x: int(x[4:]))
word['word_word_num'] = word.word_id.apply(lambda x: int(x[6:]))
word = word.merge(book, how='left', left_on='word_book_id', right_on='book_id')\
    .rename(columns={'book_name_abbrev':'word_book_name_abbrev',})
word["word_lexn_id_copy"] = word.word_lexn_id    
word["word_pars_id_copy"] = word.word_pars_id    
word = word[[
 'word_id',
 'word_greek',
 'word_english',
 'word_greek_punc',
 'word_english_punc',
 'word_book_name_abbrev',
 'word_chap_num',
 'word_vers_num',
 'word_word_num',
 'word_lexn_id_copy',
 'word_pars_id_copy',
 'word_pars_id',
 'word_lexn_id',
 'word_book_id',
 'word_chap_id',
 'word_vers_id', 
 
 ]]
word.sample(5)

In [ ]:
# chapter model
chap = df.groupby('word_chap_id').count()['wd'].reset_index().rename(columns={'wd':'chap_num_words'})
temp = df.groupby(['word_chap_id', 'vs']).count()['wd'].reset_index()
num_verses_per_chapter = temp.groupby('word_chap_id').count()['wd'].reset_index().rename(columns={'wd':'chap_num_verses'})
chap = chap.merge(num_verses_per_chapter, how='left', on='word_chap_id').rename(columns={'word_chap_id':'chap_id'})
chap['chap_num'] = chap.chap_id.apply(lambda x: int(x[2:]))
chap['chap_book_id'] = chap.chap_id.apply(lambda x: x[:2])

chap.sample(5)

In [ ]:
# verse model
vers = df.groupby('word_vers_id').count()['wd'].reset_index().rename(columns={'word_vers_id':'vers_id', 'wd':'vers_num_words'})
print(vers.vers_num_words.sum())
vers['vers_chap_num'] = vers.vers_id.apply(lambda x: int(x[2:4]))
vers['vers_num'] = vers.vers_id.apply(lambda x: int(x[4:]))
vers['vers_book_id'] = vers.vers_id.apply(lambda x: x[:2])
vers['vers_chap_id'] = vers.vers_id.apply(lambda x: x[:4])
vers = vers.merge(book, how='left', left_on='vers_book_id', right_on='book_id')
vers['vers_ref_abbrev'] = vers.apply(lambda x: x.book_name_abbrev + '.' + " " + str(x.vers_chap_num) + ":" + str(x.vers_num), axis=1)
vers['vers_ref'] = vers.apply(lambda x: x.book_name + " " + str(x.vers_chap_num) + ":" + str(x.vers_num), axis=1)
vers['vers_chap_url'] = vers.apply(lambda x: x.book_name_abbrev + "-" + str(x.vers_chap_num) + "#verse" + str(x.vers_num), axis=1)
vers = vers.rename(columns={'book_name_abbrev':'vers_book_name_abbrev'})
vers = vers[[
    'vers_id',
    'vers_ref',
    'vers_ref_abbrev',
    'vers_chap_url',
    'vers_book_name_abbrev',
    'vers_chap_num',
    'vers_num',
    'vers_num_words',
    'vers_book_id',
    'vers_chap_id',
]]
vers.sample(5)


In [ ]:
# get the freq of each lexn word in each chapter
temp = query(f"""
SELECT
    concat(word_lexn_id,'_',word_chap_id) as frlc_id, 
    word_lexn_id as frlc_lexn_id,
    word_book_id as frlc_book_id,
    word_chap_id as frlc_chap_id,
    count(*) as frlc_count
FROM word
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4
""").to_df()

frlc = temp\
    .merge(chap, how='left', left_on='frlc_chap_id', right_on='chap_id')\
    .merge(book, how='left', left_on='frlc_book_id', right_on='book_id')
frlc = frlc.rename(columns={'book_name_abbrev': 'frlc_book_name_abbrev', 'chap_num':'frlc_chap_num'})
frlc = frlc[['frlc_id', 'frlc_book_name_abbrev', 'frlc_chap_num', 'frlc_count', 'frlc_lexn_id',]]
frlc[frlc.frlc_lexn_id=="0100"].sample(10)


In [ ]:
# get the freq of each lexn word in each book
temp = query(f"""
SELECT
    concat(word_lexn_id,'_',word_book_id) as frlb_id, 
    word_lexn_id as frlb_lexn_id,
    word_book_id as frlb_book_id,    
    count(*) as frlb_count
FROM word
GROUP BY 1,2,3
ORDER BY 1,2,3
""").to_df()

frlb = temp.merge(book, how='left', left_on='frlb_book_id', right_on='book_id')
frlb = frlb.rename(columns={'book_name_abbrev': 'frlb_book_name_abbrev', })
frlb = frlb[['frlb_id', 'frlb_book_name_abbrev', 'frlb_count', 'frlb_lexn_id',]]
frlb[frlb.frlb_lexn_id=="0100"].sample(10)

In [ ]:
# combine all into one table
gnt = word\
        .merge(vers, how='left', left_on="word_vers_id", right_on="vers_id")\
        .merge(chap, how='left', left_on="word_chap_id", right_on="chap_id")\
        .merge(book, how='left', left_on="word_book_id", right_on="book_id")\
        .merge(pars, how='left', left_on="word_pars_id", right_on="pars_id")\
        .merge(lexn, how='left', left_on="word_lexn_id", right_on="lexn_id")

In [ ]:
# save pickles for django models
word.to_pickle('../../pickles/word.pkl')
pars.to_pickle('../../pickles/pars.pkl')
lexn.to_pickle('../../pickles/lexn.pkl')
pdgm.to_pickle('../../pickles/pdgm.pkl')
book.to_pickle('../../pickles/book.pkl')
chap.to_pickle('../../pickles/chap.pkl')
vers.to_pickle('../../pickles/vers.pkl')
frlc.to_pickle('../../pickles/frlc.pkl')
frlb.to_pickle('../../pickles/frlb.pkl')
gnt.to_pickle('../../pickles/gnt.pkl')

In [ ]:
# step takes about 2.75 minutes
with pd.ExcelWriter('../../../api/pickles/gnt.xlsx') as writer:  
    word.to_excel(writer, sheet_name='word')
    pars.to_excel(writer, sheet_name='pars')
    lexn.to_excel(writer, sheet_name='lexn')
    pdgm.to_excel(writer, sheet_name='pdgm')
    book.to_excel(writer, sheet_name='book')
    chap.to_excel(writer, sheet_name='chap')
    vers.to_excel(writer, sheet_name='vers')
    frlc.to_excel(writer, sheet_name='frlc')
    frlb.to_excel(writer, sheet_name='frlb')
    gnt.to_excel(writer, sheet_name='gnt')